In [10]:
import matplotlib.pyplot as plt  # plotting library
import seaborn as sns  # plotting library
import math # math
import glob
import csv
import statistics
from pathlib import Path
import scipy
import scipy.special
import numpy
import inspect
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt


In [ ]:
%run BomInfo-Util.ipynb

In [12]:
%run BomInfo-OperationsGraph.ipynb import build_operation_tree

In [19]:

def add_disjunctive_edges(G, dict_machine_operations, dict_operations_machine):
    """
    Edge (X,Y) edge between operations executed on same machine 
    """
    op_no = len(dict_operations_machine)
    for m_id, ops in dict_machine_operations.items():
        ops = list(ops)
        ops_per_machine=len(ops)
        for i in range(ops_per_machine-1):
            for j in range(i+1, ops_per_machine):
                reunion = set()
                reunion.update( dict_operations_machine[ops[i]])
                reunion.update(dict_operations_machine[ops[j]])
                intersection = dict_operations_machine[ops[i]].intersection(dict_operations_machine[ops[j]])
                G.add_edge(f'O{ops[i]}', f'O{ops[j]}', machine= f"M{m_id}", 
                           m_operations = ops_per_machine / op_no, 
                           m_shared_intersection = len(intersection) / ops_per_machine, 
                           m_shared_reunion = len(reunion) / ops_per_machine)
        
def build_auxilary_structures_for_disjunctive_edges(current_node, product_machine_dictionary, dict_machine_operations, dict_operations_machine):
    operation_id = current_node['operationid']
    product_id = current_node['productid']
    
    machines = product_machine_dictionary[product_id]
            
    for m in machines:
        m_id = m['id']
        if m_id not in dict_machine_operations:
            dict_machine_operations[m_id] = {operation_id}
        else:
            dict_machine_operations[m_id].add(operation_id)

        if operation_id not in  dict_operations_machine:
            dict_operations_machine[operation_id] = {m_id}
        else:
            dict_operations_machine[operation_id].add(m_id)  

    if 'children' not in current_node:
        return
    
    children = current_node['children']
    if len(children) == 0:
        return

    for node in children:
        build_auxilary_structures_for_disjunctive_edges(node, product_machine_dictionary, dict_machine_operations, dict_operations_machine)


def build_disjunctive_graph(current_node, G, product_machine_dictionary):
    """
    Node = operation
       - information = no. of machines on which can be produced
    Edge precedence relation betwwen operations(A,B) 
       - information = no. of B pices to produce A
    Edge (X,Y) edge between operations executed on same machine 
    """
    build_operation_tree(current_node, G, product_machine_dictionary)
    
    dict_machine_operations = {}
    dict_operations_machine = {}
    build_auxilary_structures_for_disjunctive_edges(current_node, product_machine_dictionary, dict_machine_operations, dict_operations_machine)
    add_disjunctive_edges(G, dict_machine_operations, dict_operations_machine)
    

In [20]:
def build_disjuctive_graph_charactristics(graphs):
   
    data_problem  = build_operation_graph_characteristics(graphs)

    data_problem['machine_per_operation_edges'] = []

    for g in graphs:
        no_of_machine_edges = len([(u,v) for u,v,e in g.edges(data=True) if ('machine' in e) and e['machine'][0] == 'M'])
        data_problem['machine_per_operation_edges'].append((g.number_of_edges()-no_of_machine_edges)/no_of_machine_edges)

    return data_problem


In [21]:
#CONFIGURATIONS
out_path_dir='../datasets/results/instances_characteristics/ianuarie2025-v2/disjunctive'
dataset_path_dir='../datasets/'
weight_atts = [None,'quantity', 's_execution_time', 's_quantity', 's_number',
               'm_operations', 'm_shared_intersection', 'm_shared_reunion']


In [22]:
def buildDatasetDisjunctive(dataset_graph, output_file_name):
    """
    :param: dataset_graph - list of disjunctive multigraphs
    :param: output_file_name - file name to store characteristics
    """
    build_charactristics_info(dataset_graph, build_disjuctive_graph_charactristics, 
                              out_path_dir, output_file_name, append_to_file=False, 
                              weight_atts=weight_atts)


In [23]:
def buildALL():
    """
    Build operation graph characteristics for all test  instances
    """
    #load like multigraphs
    _2asp_graphs, dyuthi_graphs, fjssp_graph, asp_deep_graphs, asp_wide_graphs, asp_mixed_graphs, dafjs_graphs, yfjs_graphs = load_graphs(build_disjunctive_graph)

    # buildDatasetDisjunctive(_2asp_graphs, 'bom_info_graph_2asp.csv')
    #buildDatasetDisjunctive(fjssp_graph, 'bom_info_graph_fjssp.csv')
    #buildDatasetDisjunctive(dyuthi_graphs, 'bom_info_graph_dyuthi.csv')
    #buildDatasetDisjunctive(asp_wide_graphs, 'bom_info_graph_wide.csv')
    #buildDatasetDisjunctive(asp_deep_graphs, 'bom_info_graph_deep.csv')
    #buildDatasetDisjunctive(asp_mixed_graphs, 'bom_info_graph_mixed.csv')
    # buildDatasetDisjunctive(dafjs_graphs, 'bom_info_graph_dafjs.csv')
    buildDatasetDisjunctive(yfjs_graphs, 'bom_info_graph_yfjs.csv')
#buildALL()

In [24]:
buildALL()

KeyboardInterrupt: 

# Graph Visualisation

In [10]:
#display
from pyvis import network as pvnet

def plot_g_pyviz(G, name='out.html', height='300px', width='700px'):
    g = G.copy() # some attributes added to nodes
    net = pvnet.Network(notebook=True, directed=True, height=height, width=width)
    opts = '''
        var options = {
          "physics": {
            "forceAtlas2Based": {
              "gravitationalConstant": -100,
              "centralGravity": 0.11,
              "springLength": 100,
              "springConstant": 0.09,
              "avoidOverlap": 1
            },
            "minVelocity": 0.75,
            "solver": "forceAtlas2Based",
            "timestep": 0.22
          }
        }
    '''

    net.set_options(opts)
    # uncomment this to play with layout
    # net.show_buttons(filter_=['physics'])
    net.from_nx(g)
    for edge in net.edges:
         #print(edge)
        if 'machine' in edge and isinstance(edge['machine'],str) and edge['machine'][0] == 'M':
            edge['color']='green'
       
    for node in net.nodes:
        #print(node)
        if isinstance(node['label'],str) and node['label'][0] == 'M':
            node['color']='red'
            node['shape']='box'
        #print(node)
    return net.show(name)

In [11]:
plot_g_pyviz(dafjs[0])

NameError: name 'dafjs' is not defined